In [1]:
import pandas as pd
import numpy as np
import spacy
import time
import dask.dataframe as dd

In [2]:
df = pd.read_csv('used_cars_data.csv', nrows=1000)

In [3]:
# Load the data into Dask using the type map
df = dd.from_pandas(df, npartitions=3)

In [4]:
df.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [5]:
colors = df['exterior_color']

In [6]:
colors

Dask Series Structure:
npartitions=3
0      object
334       ...
668       ...
999       ...
Name: exterior_color, dtype: object
Dask Name: getitem, 6 tasks

In [9]:
nlp = spacy.load('en')

def tokenizer(text):
    stripped = ' '.join(text.exterior_color.lower().split())
    tokens = nlp(stripped)
    return ' '.join([token.lemma_ for token in tokens])

#def dask_tokenizer(row):    

In [ ]:
def printer(df):

In [10]:
colors.map_partitions(tokenizer)

ValueError: Metadata inference failed in `tokenizer`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
AttributeError("'Series' object has no attribute 'exterior_color'")

Traceback:
---------
  File "/home/owen/bin/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py", line 174, in raise_on_meta_error
    yield
  File "/home/owen/bin/anaconda3/lib/python3.8/site-packages/dask/dataframe/core.py", line 5165, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "<ipython-input-9-ce308388fdcc>", line 4, in tokenizer
    stripped = ' '.join(text.exterior_color.lower().split())
  File "/home/owen/bin/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py", line 5139, in __getattr__
    return objec

In [ ]:
colors = colors.apply(lambda x : ' '.join(x.lower().split()))

In [ ]:
colors = colors.apply(lambda x : nlp(x), meta=('exterior_color', 'object'))

In [ ]:
colors = colors.apply(lambda x : [token.lemma_ for token in x])

In [ ]:
colors = colors.apply(lambda x : ' '.join(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word')

In [ ]:
X = vectorizer.fit_transform(colors)

In [ ]:
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
bow_df

In [ ]:
bow_df.describe().sort_values(by='mean', axis=1, ascending=False)